## 1.1 Instalando Dependências

In [1]:
!pip install pymongo

   ---------------------------------------- 0.0/955.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/955.1 kB ? eta -:--:--
   ---------- ----------------------------- 262.1/955.1 kB ? eta -:--:--
   ---------- ----------------------------- 262.1/955.1 kB ? eta -:--:--
   -------------------- ----------------- 524.3/955.1 kB 741.9 kB/s eta 0:00:01
   -------------------- ----------------- 524.3/955.1 kB 741.9 kB/s eta 0:00:01
   ------------------------------- ------ 786.4/955.1 kB 754.8 kB/s eta 0:00:01
   ------------------------------- ------ 786.4/955.1 kB 754.8 kB/s eta 0:00:01
   ------------------------------- ------ 786.4/955.1 kB 754.8 kB/s eta 0:00:01
   -------------------------------------- 955.1/955.1 kB 476.1 kB/s eta 0:00:00

   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dn

In [19]:
!pip install pandas

In [20]:
import pymongo
import pandas as pd

## 1.2.	Criação do Pandas Dataframe

In [33]:
dados_carros = {
    'Carro': ['Onix','Polo','Sandero','Fiesta','City'],
    'Cor': ['Prata','Branco','Prata','Vermelho','Preto'],
    'Montadora': ['Chevrolet','Volkswagen','Renault','Ford','Honda']
}
df_carros = pd.DataFrame(dados_carros)
df_carros.describe()

,Carro,Cor,Montadora
count,5,5,5
unique,5,4,5
top,Onix,Prata,Chevrolet
freq,1,2,1


In [34]:
dados_montadoras = {
    'Montadora': ['Chevrolet','Volkswagen','Renault','Ford','Honda'],
    'País': ['EUA','Alemanha','França','EUA','Japão']
}
df_montadoras = pd.DataFrame(dados_montadoras)
df_montadoras.describe()

,Montadora,País
count,5,5
unique,5,4
top,Chevrolet,EUA
freq,1,2


## 1.3.	Salvar Pandas Dataframe no MongoDB

In [27]:
client = pymongo.MongoClient("mongodb://localhost:27017")
local_database = client['local']

In [28]:
Carros_collection = local_database['Carros']

In [30]:
Montadoras_collection = local_database['Montadoras']

In [31]:
Carros_collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'local'), 'Carros')

In [32]:
Montadoras_collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'local'), 'Montadoras')

In [36]:
df_carros.to_dict(orient="records")

[{'Carro': 'Onix', 'Cor': 'Prata', 'Montadora': 'Chevrolet'},
 {'Carro': 'Polo', 'Cor': 'Branco', 'Montadora': 'Volkswagen'},
 {'Carro': 'Sandero', 'Cor': 'Prata', 'Montadora': 'Renault'},
 {'Carro': 'Fiesta', 'Cor': 'Vermelho', 'Montadora': 'Ford'},
 {'Carro': 'City', 'Cor': 'Preto', 'Montadora': 'Honda'}]

In [37]:
Carros_collection.insert_many(df_carros.to_dict(orient="records"))

InsertManyResult([ObjectId('68782b1b944b77d2780a2feb'), ObjectId('68782b1b944b77d2780a2fec'), ObjectId('68782b1b944b77d2780a2fed'), ObjectId('68782b1b944b77d2780a2fee'), ObjectId('68782b1b944b77d2780a2fef')], acknowledged=True)

In [39]:
df_montadoras.to_dict(orient="records")

[{'Montadora': 'Chevrolet', 'País': 'EUA'},
 {'Montadora': 'Volkswagen', 'País': 'Alemanha'},
 {'Montadora': 'Renault', 'País': 'França'},
 {'Montadora': 'Ford', 'País': 'EUA'},
 {'Montadora': 'Honda', 'País': 'Japão'}]

In [40]:
Montadoras_collection.insert_many(df_montadoras.to_dict(orient="records"))

InsertManyResult([ObjectId('68782b5d944b77d2780a2ff0'), ObjectId('68782b5d944b77d2780a2ff1'), ObjectId('68782b5d944b77d2780a2ff2'), ObjectId('68782b5d944b77d2780a2ff3'), ObjectId('68782b5d944b77d2780a2ff4')], acknowledged=True)

In [41]:
Montadoras_collection.find()[0]

{'_id': ObjectId('68782b5d944b77d2780a2ff0'),
 'Montadora': 'Chevrolet',
 'País': 'EUA'}

## 1.4.	Criando Agregação no MongoDB

In [65]:
# dict de LOOKUP
# seleção de carros realizando lookup em montadoras
montadoras_lookup = {
    "$lookup": {
        "from": "Montadoras",
        "localField": "Montadora",
        "foreignField": "Montadora",
        "as": "Montadoras",
    }
}

projecao = {
    "$project": {
        "_id": 1,
        "Carro": 1,
        "Cor": 1,
        "Montadora": 1,
        "Montadoras": 1,
        "Pais":{
            "$arrayElemAt": ["$Montadoras.País", 0]
        }
    }
}

# Montar o pipeline
pipeline_lookup = [montadoras_lookup, projecao]

# Puxar um aggregate do collection passando a pipeline de LOOKUP
resultados_agregacao = Carros_collection.aggregate(pipeline_lookup)
resultados_agregacao_list = list(resultados_agregacao)

# Mostrar resultados
df_resultados_agregacao = pd.DataFrame(resultados_agregacao_list)
df_resultados_agregacao

,_id,Carro,Cor,Montadora,Montadoras,Pais
0,68782b1b944b77d2780a2feb,Onix,Prata,Chevrolet,"[{'_id': 68782b5d944b77d2780a2ff0, 'Montadora'...",EUA
1,68782b1b944b77d2780a2fec,Polo,Branco,Volkswagen,"[{'_id': 68782b5d944b77d2780a2ff1, 'Montadora'...",Alemanha
2,68782b1b944b77d2780a2fed,Sandero,Prata,Renault,"[{'_id': 68782b5d944b77d2780a2ff2, 'Montadora'...",França
3,68782b1b944b77d2780a2fee,Fiesta,Vermelho,Ford,"[{'_id': 68782b5d944b77d2780a2ff3, 'Montadora'...",EUA
4,68782b1b944b77d2780a2fef,City,Preto,Honda,"[{'_id': 68782b5d944b77d2780a2ff4, 'Montadora'...",Japão


In [66]:
resultados_agregacao_list

[{'_id': ObjectId('68782b1b944b77d2780a2feb'),
  'Carro': 'Onix',
  'Cor': 'Prata',
  'Montadora': 'Chevrolet',
  'Montadoras': [{'_id': ObjectId('68782b5d944b77d2780a2ff0'),
    'Montadora': 'Chevrolet',
    'País': 'EUA'}],
  'Pais': 'EUA'},
 {'_id': ObjectId('68782b1b944b77d2780a2fec'),
  'Carro': 'Polo',
  'Cor': 'Branco',
  'Montadora': 'Volkswagen',
  'Montadoras': [{'_id': ObjectId('68782b5d944b77d2780a2ff1'),
    'Montadora': 'Volkswagen',
    'País': 'Alemanha'}],
  'Pais': 'Alemanha'},
 {'_id': ObjectId('68782b1b944b77d2780a2fed'),
  'Carro': 'Sandero',
  'Cor': 'Prata',
  'Montadora': 'Renault',
  'Montadoras': [{'_id': ObjectId('68782b5d944b77d2780a2ff2'),
    'Montadora': 'Renault',
    'País': 'França'}],
  'Pais': 'França'},
 {'_id': ObjectId('68782b1b944b77d2780a2fee'),
  'Carro': 'Fiesta',
  'Cor': 'Vermelho',
  'Montadora': 'Ford',
  'Montadoras': [{'_id': ObjectId('68782b5d944b77d2780a2ff3'),
    'Montadora': 'Ford',
    'País': 'EUA'}],
  'Pais': 'EUA'},
 {'_id': Ob

## 1.5. Agrupando informações

In [67]:
# definir dict de LOOKUP
# selecionar carros realizando LOOKUP em montadora

# dict de LOOKUP
# seleção de carros realizando lookup em montadoras
montadoras_lookup = {
    "$lookup": {
        "from": "Montadoras",
        "localField": "Montadora",
        "foreignField": "Montadora",
        "as": "Montadoras",
    }
}

projecao = {
    "$project": {
        "_id": 1,
        "Carro": 1,
        "Cor": 1,
        "Montadora": 1,
        "Montadoras": 1,
        "Pais":{
            "$arrayElemAt": ["$Montadoras.País", 0]
        }
    }
}

agrupamento = {
    "$group": {
        "_id": "$Pais",
        "Carros": {
            "$push": {
                "Carro": "$Carro", "Cor": "$Cor", "Montadora": "$Montadora"
            }
        }
    }
}

# Montar o pipeline
pipeline_lookup = [montadoras_lookup, projecao, agrupamento]

# Puxar um aggregate do collection passando a pipeline de LOOKUP
resultados_agregacao_group = Carros_collection.aggregate(pipeline_lookup)
resultados_agregacao_list_group = list(resultados_agregacao_group)

# Mostrar resultados
df_resultados_agregacao_group = pd.DataFrame(resultados_agregacao_list_group)
df_resultados_agregacao_group

,_id,Carros
0,EUA,"[{'Carro': 'Onix', 'Cor': 'Prata', 'Montadora'..."
1,França,"[{'Carro': 'Sandero', 'Cor': 'Prata', 'Montado..."
2,Alemanha,"[{'Carro': 'Polo', 'Cor': 'Branco', 'Montadora..."
3,Japão,"[{'Carro': 'City', 'Cor': 'Preto', 'Montadora'..."


In [69]:
resultados_agregacao_list_group

[{'_id': 'EUA',
  'Carros': [{'Carro': 'Onix', 'Cor': 'Prata', 'Montadora': 'Chevrolet'},
   {'Carro': 'Fiesta', 'Cor': 'Vermelho', 'Montadora': 'Ford'}]},
 {'_id': 'França',
  'Carros': [{'Carro': 'Sandero', 'Cor': 'Prata', 'Montadora': 'Renault'}]},
 {'_id': 'Alemanha',
  'Carros': [{'Carro': 'Polo', 'Cor': 'Branco', 'Montadora': 'Volkswagen'}]},
 {'_id': 'Japão',
  'Carros': [{'Carro': 'City', 'Cor': 'Preto', 'Montadora': 'Honda'}]}]